<a href="https://colab.research.google.com/github/hhmida/ESB/blob/main/Exercice_SPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 40 kB/s 
     |████████████████████████████████| 199 kB 57.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=d95a02b9291d0b9b5ae57188e27d68b193f49340da14549791c9f7779902d9cf
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
!wget https://hhmida.github.io/tp-spark/assets/vente_export_der_ph_mois.csv
!wget https://hhmida.github.io/tp-spark/assets/prod_der_phosphate_mois.csv

--2022-12-02 09:39:18--  https://hhmida.github.io/tp-spark/assets/vente_export_der_ph_mois.csv
Resolving hhmida.github.io (hhmida.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to hhmida.github.io (hhmida.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6039 (5.9K) [text/csv]
Saving to: ‘vente_export_der_ph_mois.csv’

vente_export_der_ph 100%[===================>]   5.90K  --.-KB/s    in 0s      

2022-12-02 09:39:18 (52.3 MB/s) - ‘vente_export_der_ph_mois.csv’ saved [6039/6039]

--2022-12-02 09:39:18--  https://hhmida.github.io/tp-spark/assets/prod_der_phosphate_mois.csv
Resolving hhmida.github.io (hhmida.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to hhmida.github.io (hhmida.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7434 (7.3K) [text/csv]
Saving to: ‘prod_der_phosphate_mois.csv’

prod_der_phosphate_ 100%[==

In [6]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark =  SparkSession.builder.appName('Exercice Spark').getOrCreate()
sc = spark.sparkContext

In [26]:
from itertools import islice

In [20]:
rddP = sc.textFile('prod_der_phosphate_mois.csv').cache()
rddP = rddP.mapPartitionsWithIndex(lambda idx, iter: islice(iter,1,None) if (idx == 0) else iter)


In [24]:
rddV = sc.textFile('vente_export_der_ph_mois.csv').cache()
rddV = rddV.mapPartitionsWithIndex(lambda idx, iter: islice(iter,1,None) if (idx == 0) else iter)

In [36]:
def transformer(l):
  T=l.split(';')
  return int(T[0]),T[-1]

In [40]:
rddP.map(transformer).sortByKey(False).collect()[0]

(8030, '31/08/2011')

In [41]:
def fmax(a,b):
  if a>b:
    return a
  return b

In [49]:
rddP.map(lambda l:(int(l.split(';')[0]),l.split(';')[-1])).reduce(lambda a,b:a if a>b else b)

(8030, '31/08/2011')